In [10]:
import json
import pandas as pd
import numpy as np
from pymystem3 import Mystem

# !pip3 install spacy 
# !python -m spacy download ru_core_news_sm
import spacy
import re
from string import punctuation
import random
import ast

In [11]:
mystem = Mystem()
def lemm_phrase(x):
    tokens = mystem.lemmatize(x)
    tokens = [token for token in tokens[:-1] if (token.strip() not in [" ", ""])]
    return " ".join(tokens)

# Work with generated data

In [ ]:
df = pd.read_csv("dataset/generated_data.csv")

In [ ]:
pattern = re.compile(r'"|\'|\.|,|\\|\/|-|\(|\)|,')

df['new_text'] = df['text'].apply(lambda x: re.sub(pattern=pattern, repl='', string=x.lower()).strip())
df['text'] = df['text'].apply(lambda x: re.sub(pattern=pattern, repl='', string=x).strip())

In [ ]:
df['new_text'] = df['new_text'].apply(lambda x: lemm_phrase(x))

In [ ]:
df

In [ ]:
# df.to_csv('dataset/final_data_lower.csv', index=False)

# Read New Dataset

In [3]:
df = pd.read_csv("dataset/final_data_lower.csv")
df = df[:390]

In [4]:
pattern = re.compile(r':')
df['text'] = df['text'].apply(lambda x: re.sub(pattern=pattern, repl=' : ', string=x).strip())
df['new_text'] = df['new_text'].apply(lambda x: re.sub(pattern=pattern, repl=' : ', string=x).strip())

In [5]:
df['new_text'] = df['new_text'].apply(lambda x: ' '.join([i for i in x.split() if i != '']))
df['text'] = df['text'].apply(lambda x: ' '.join([i for i in x.split() if i != '']))

In [6]:
df

,text,new_text
0,Создай задачу : Подготовить отчет к 15 : 00 за...,создавать задача : подготовить отчет к 15 : 00...
1,Создай задание : Отправить приглашения на собр...,создавать задание : отправлять приглашение на ...
2,Создай задачу : Разработать презентацию к 18 :...,создавать задача : разрабатывать презентация к...
3,Назначь обязанность : Провести собрание с кома...,назначать обязанность : проводить собрание с к...
4,Поставь задачу : Подготовить план мероприятий ...,поставлять задача : подготовить план мероприят...
...,...,...
385,Создай задачу : Виктор Романов подготовьте отч...,создавать задача : виктор роман подготовить от...
386,Создай задачу : Елене Савельевой нужно разрабо...,создавать задача : елена савельева нужно разра...
387,Создай задачу : Попросить Ивана Иванова подгот...,создавать задача : попросить иван иванов подго...
388,Создай задачу : Напомнить Маше завтра в 10 утр...,создавать задача : напоминать маша завтра в 10...


# Use spacy

In [7]:
from spacy.lang.ru import Russian
from spacy.pipeline import EntityRuler

In [8]:
tasks = [
    'создавать задача',
    'создавать заметка',
    'создавать задание',
    'создавать поручение',
    'напоминать',
    'запланировать встреча',
    'назначать обязанность',
    'ставить задача',
    'задавать задача',
    'давать поручение',
    'давать задача',
    'добавлять задача',
    'поставлять задача'
]

tags = {
    "Task": 'task_type',
    "ToDo": 'todo',
    "Person": 'person',
    "Time": 'time',
    "Garbage": 'garbage'
}

In [9]:
def create_training_data(data, type):
    data = data
    patterns = []
    for item in data:
        pattern = {"label": type, "pattern": item}
        # generate list of patterns
        patterns.append(pattern)
    return patterns

In [10]:
nlp = Russian()
ruler = EntityRuler(nlp)
ruler = nlp.add_pipe("entity_ruler")
def generate_rules(patterns, name):
    ruler.add_patterns(patterns)
    # nlp.to_disk(name)

In [11]:
def test_model(model, text):
    doc = model(text)
    results = []
    entities = []

    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) > 0:
        results = [text, {"entities": entities}]
        return results
"""
TRAIN DATA FOR SPACY = [
        (   text, 
            {"entities": 
                [(start, end, label), ...]
            }
        )
    ]
"""

'\nTRAIN DATA FOR SPACY = [\n        (   text, \n            {"entities": \n                [(start, end, label), ...]\n            }\n        )\n    ]\n'

In [12]:
def get_train_type(model, x, arr):
    results = test_model(model, x)
    if results != None:
        arr.append(results)

# Выделение типа задачи

In [13]:
task_patterns = create_training_data(tasks, tags['Task'])
generate_rules(patterns=task_patterns, name='task_types')

# Выделение имен

In [ ]:
# df_fem = pd.read_csv('dataset/female_names.csv')
# df_male = pd.read_csv('dataset/male_names.csv')
# df_names = df_fem.append(df_male)
# df_names.to_csv('dataset/names.csv', index=False)

In [ ]:

# df_names['name'] = df_names['name'].apply(lambda x: x.lower())
# df_names['name'] = df_names['name'].apply(lambda x: lemm_phrase(x))

In [14]:
df_names = pd.read_csv('dataset/names.csv')
person_patterns = create_training_data(list(df_names['name']), tags['Person'])
generate_rules(patterns=person_patterns, name='person')

# Выделение времени

In [15]:
df_time = pd.read_csv('dataset/with_months.csv')

In [ ]:
# df_time.to_csv('dataset/time.csv', index=False)

In [ ]:
# df_time['time'] = df_time['time'].apply(lambda x: x.lower())
# df_time['lower_time'] = df_time['time'].apply(lambda x: lemm_phrase(x))

In [16]:
time_patterns = create_training_data(list(df_time['lower_time']), tags['Time'])
generate_rules(patterns=time_patterns, name='time')

# Model with all entities

In [ ]:
# nlp.to_disk('final_all_entities')

# Train Data

In [17]:
TRAIN_DATA = []
df['new_text'].apply(lambda x: get_train_type(nlp, x, TRAIN_DATA))

0      None
1      None
2      None
3      None
4      None
       ... 
385    None
386    None
387    None
388    None
389    None
Name: new_text, Length: 390, dtype: object

In [18]:
from spacy.training.example import Example

In [19]:
def train(data, epochs=30):
    TRAIN_DATA = data
    nlp = spacy.blank("ru")
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe("ner", last=True)

    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2]) # taking label for ex. person

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itr in range(epochs):
            print("Start Epoch - " + str(itr))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    nlp.update(
                        [example],
                        drop=0.2,
                        sgd=optimizer,
                        losses=losses
                    )
            print(losses)
    return nlp


In [20]:
nlp = train(TRAIN_DATA, epochs=50)
nlp.to_disk('final_4_for_test')

Start Epoch - 0
{'ner': 507.5993332595376}
Start Epoch - 1
{'ner': 61.789769965887466}
Start Epoch - 2
{'ner': 34.166370042529934}
Start Epoch - 3
{'ner': 50.1018232148208}
Start Epoch - 4
{'ner': 37.930383771571705}
Start Epoch - 5
{'ner': 10.466288085141624}
Start Epoch - 6
{'ner': 22.10927633053586}
Start Epoch - 7
{'ner': 15.97158817384445}
Start Epoch - 8
{'ner': 24.572980784801043}
Start Epoch - 9
{'ner': 32.75897525183957}
Start Epoch - 10
{'ner': 11.550055545625863}
Start Epoch - 11
{'ner': 30.184005635220405}
Start Epoch - 12
{'ner': 5.686790370003248}
Start Epoch - 13
{'ner': 19.389103707826415}
Start Epoch - 14
{'ner': 11.994582516970818}
Start Epoch - 15
{'ner': 10.531406020511758}
Start Epoch - 16
{'ner': 7.7893354423152426}
Start Epoch - 17
{'ner': 6.910647088298205}
Start Epoch - 18
{'ner': 5.536938580674887}
Start Epoch - 19
{'ner': 9.431434425032766}
Start Epoch - 20
{'ner': 11.851990711229805}
Start Epoch - 21
{'ner': 4.721186140612571}
Start Epoch - 22
{'ner': 20.376

# Use model

In [12]:
def clean_str(phrase):
    pattern = re.compile(r'"|\'|\.|,|\\|\/|-|\(|\)|,')
    phrase = re.sub(pattern=pattern, repl='', string=phrase.lower()).strip()
    final = lemm_phrase(phrase)
    return final

In [13]:
# nlp.to_disk('result_final_all')
nlp = spacy.load('final_4_for_test')

In [14]:
from stop_words import get_stop_words
stop_words = get_stop_words('ru')
stop_words = stop_words[:200]

In [15]:
def check_for_useless_words(text):
    # убрать лишние предлоги в конце
    count = 0
    count_back = 0
    for i in text:
        if i in stop_words:
            count += 1
        else:
            break

    for i in text[::-1]:
        if i in stop_words:
            count_back += 1
        else:
            break

    return text[count:len(text)-count_back]

In [16]:
# Вывод результата к красивом виде
def get_results(text):
    result = []
    idx = []
    for_metrics = []

    # подготовка фразы
    cleaned = clean_str(text)
    tmp = list(zip(text.split(), cleaned.replace(' : ', ':').split()))
    doc = nlp(cleaned)
    for ent in doc.ents:
        words = cleaned[ent.start_char:ent.end_char].split()
        for_metrics.append(cleaned[ent.start_char:ent.end_char])
        res = []
        for w in words:
            for t, i in enumerate(tmp):
                if w in i[1]:
                    idx.append(t)
                    res.append(i[0])
        result.append((' '.join(res), ent.label_))

    # получаем "название для задачи" - то, что не разметилось = само задание
    task = [k[0] for t, k in enumerate(tmp) if t not in idx]

    # объединяем выделенные объекты
    result = sorted(result, key=lambda x: x[1])
    final = {}
    key = None
    for i in result:
        if i[1] != key:
            key = i[1]
            final[key] = [i[0]]
        else:
            if i[0] not in final[key]:
                final[key].append(i[0])
    task = check_for_useless_words(task)
    task = ' '.join(task)
    final['task'] = task
    return final, for_metrics

In [17]:
text = 'Создай задачу: Написать сценарий для новогоднего видео до 1 декабря'
print(get_results(text)[0])

{'task_type': ['Создай задачу:'], 'time': ['1', 'декабря'], 'task': 'Написать сценарий для новогоднего видео'}


In [18]:
test = pd.read_csv('dataset/after_spacy.csv')[390:]
test['task_type'] = test['task_type'].apply(lambda x: ast.literal_eval(x))
test['person'] = test['person'].apply(lambda x: ast.literal_eval(x))
test['time'] = test['time'].apply(lambda x: ast.literal_eval(x))

In [19]:
test['all_entities'] = test.apply(lambda x: x['task_type'] + x['person'] + x['time'], axis=1)

In [20]:
test

,text,new_text,task_type,person,time,all_entities
390,Создай задачу : Петр требуется верстка лендинг...,создавать задача : петр требоваться верстка ле...,[создавать задача],[],"[2, день]","[создавать задача, 2, день]"
391,Создай задачу : Олег нужны эскизы интерьера оф...,создавать задача : олег нужный эскиз интерьер ...,[создавать задача],[олег],[],"[создавать задача, олег]"
392,Создай задачу : Анна пожалуйста разработай нес...,создавать задача : анна пожалуйста разрабатыва...,[создавать задача],[анна],"[день, 2, апрель]","[создавать задача, анна, день, 2, апрель]"
393,Создай задачу : Максим требуется дизайн нового...,создавать задача : максим требоваться дизайн н...,[создавать задача],[максим],"[15, декабрь]","[создавать задача, максим, 15, декабрь]"
394,Создай задачу : Напомнить Кате завтра приступи...,создавать задача : напоминать катя завтра прис...,"[создавать задача, напоминать]",[],[завтра],"[создавать задача, напоминать, завтра]"
...,...,...,...,...,...,...
482,Создай задачу : Провести конкурс ко Дню космон...,создавать задача : проводить конкурс ко день к...,[создавать задача],[],"[день, 12, апрель]","[создавать задача, день, 12, апрель]"
483,Создай задачу : Разработать 10 вариантов облож...,создавать задача : разрабатывать 10 вариант об...,[создавать задача],[],[10],"[создавать задача, 10]"
484,Создай задачу : Придумать 20 идей ежедневных р...,создавать задача : придумывать 20 идея ежеднев...,[создавать задача],[],"[20, месяц, 2]","[создавать задача, 20, месяц, 2]"
485,Создай задачу : Спланировать съемки для 10 рец...,создавать задача : спланировать съемка для 10 ...,[создавать задача],[],"[10, 8, март]","[создавать задача, 10, 8, март]"


In [21]:
def get_f1(marked_elems, model_found):
    TP = len([i for i in model_found if i in marked_elems])
    FP = len([i for i in model_found if i not in marked_elems])
    FN = len([i for i in marked_elems if i not in model_found])

    precision = TP/(TP + FP)
    recall = TP/(TP + FN)

    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [22]:
def get_scored_results(df):
    f1_scores = []
    for item in df.values:
        formatted_res, marked_str = get_results(item[0])
        # print(formatted_res)
        f1_scores.append(get_f1(item[5], marked_str))

    print(sum(f1_scores)/len(f1_scores))

In [23]:
get_scored_results(test)

0.9060412668660088
